In [1]:
import mysql.connector
import pandas as pd
import chardet

/tmp/ipykernel_7412/2480075298.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
df = pd.read_csv('../../data/sandwich.csv', encoding='UTF-8-SIG')
df.head()

,Rank,Cafe,Menu,Price,Address
0,1,Old Oak Tap,BLT,10.0,2109 W. Chicago Ave
1,2,Au Cheval,Fried Bologna,9.0,800 W. Randolph St
2,3,Xoco,Woodland Mushroom,9.5,445 N. Clark St
3,4,Al’s Deli,Roast Beef,9.4,914 Noyes St
4,5,Publican Quality Meats,PB&L,10.0,825 W. Fulton Mkt


In [3]:
def detect_csv_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
        return result['encoding']


csv_file_path = '../../data/Seoul_CCTV.csv'  # 파일 경로를 실제 파일의 경로로 바꿔주세요
encoding = detect_csv_encoding(csv_file_path)

print(f"The encoding of the CSV file is: {encoding}")

The encoding of the CSV file is: UTF-8-SIG


In [4]:
remote = mysql.connector.connect(
    host = 'database-1.cb2iuock61zt.ap-northeast-2.rds.amazonaws.com',
    port = 3306,
    user =  'root',
    password = 'asdf1234',
    database = 'amrdb'
)

In [8]:
cur = remote.cursor()
cur.execute("CREATE TABLE sandwich ( rand int , cafe varchar(32),menu varchar(32), price float, address varchar(32))")#숫자로 시작하는 열 이름을 backtick (`)로 감싸야 합니다.

In [10]:
sql = """insert into sandwich values (%s,%s,%s,%s,%s)""" #따옴표를 세 개씩 (""") 사용하는 것은 여러 줄에 걸친 문자열을 정의하는 Python의 표준 문법
cursor = remote.cursor(buffered=True)

In [11]:
for i , row in df.iterrows(): #iterrows()는 DataFrame의 각 행을 (인덱스, 행 데이터)의 튜플로 반환
    cursor.execute(sql, tuple(row)) # execute 메서드를 호출하여 SQL 쿼리를 실행하고, 튜플의 값이 쿼리에 매개변수로 전달되어 데이터베이스에 레코드를 추가
    print(tuple(row))
    remote.commit()

(1, 'Old Oak Tap', 'BLT', 10.0, '2109 W. Chicago Ave')
(2, 'Au Cheval', 'Fried Bologna', 9.0, '800 W. Randolph St')
(3, 'Xoco', 'Woodland Mushroom', 9.5, ' 445 N. Clark St')
(4, 'Al’s Deli', 'Roast Beef', 9.4, ' 914 Noyes St')
(5, 'Publican Quality Meats', 'PB&L', 10.0, '825 W. Fulton Mkt')
(6, 'Hendrickx Belgian Bread Crafter', 'Belgian Chicken Curry Salad', 7.25, ' 100 E. Walton St')
(7, 'Acadia', 'Lobster Roll', 16.0, '1639 S. Wabash Ave')
(8, 'Birchwood Kitchen', 'Smoked Salmon Salad', 10.0, '2211 W. North Ave')
(9, 'Cemitas Puebla', 'Atomica Cemitas', 9.0, '3619 W. North Ave')
(10, 'Nana', 'Grilled Laughing Bird Shrimp and Fried Po’ Boy', 17.0, '3267 S. Halsted St')
(11, 'Lula Cafe', 'Ham and Raclette Panino', 11.0, '2537 N. Kedzie Blvd')
(12, 'Ricobene’s', 'Breaded Steak', 5.49, ' Multiple location')
(13, 'Frog n Snail', 'The Hawkeye', 14.0, '3124 N. Broadwa')
(14, 'Crosby’s Kitchen', 'Chicken Dip', 10.0, '3455 N. Southport Ave')
(15, 'Longman & Eagle', 'Wild Boar Sloppy Joe', 13

In [12]:
cursor.execute("select * from sandwich")

result = cursor.fetchall()
for row in result:
    print(row)

(1, 'Old Oak Tap', 'BLT', 10.0, '2109 W. Chicago Ave')
(2, 'Au Cheval', 'Fried Bologna', 9.0, '800 W. Randolph St')
(3, 'Xoco', 'Woodland Mushroom', 9.5, ' 445 N. Clark St')
(4, 'Al’s Deli', 'Roast Beef', 9.4, ' 914 Noyes St')
(5, 'Publican Quality Meats', 'PB&L', 10.0, '825 W. Fulton Mkt')
(6, 'Hendrickx Belgian Bread Crafter', 'Belgian Chicken Curry Salad', 7.25, ' 100 E. Walton St')
(7, 'Acadia', 'Lobster Roll', 16.0, '1639 S. Wabash Ave')
(8, 'Birchwood Kitchen', 'Smoked Salmon Salad', 10.0, '2211 W. North Ave')
(9, 'Cemitas Puebla', 'Atomica Cemitas', 9.0, '3619 W. North Ave')
(10, 'Nana', 'Grilled Laughing Bird Shrimp and', 17.0, '3267 S. Halsted St')
(11, 'Lula Cafe', 'Ham and Raclette Panino', 11.0, '2537 N. Kedzie Blvd')
(12, 'Ricobene’s', 'Breaded Steak', 5.49, ' Multiple location')
(13, 'Frog n Snail', 'The Hawkeye', 14.0, '3124 N. Broadwa')
(14, 'Crosby’s Kitchen', 'Chicken Dip', 10.0, '3455 N. Southport Ave')
(15, 'Longman & Eagle', 'Wild Boar Sloppy Joe', 13.0, '2657 N. K

In [13]:
df = pd.DataFrame(result)
df.head()

,0,1,2,3,4
0,1,Old Oak Tap,BLT,10.0,2109 W. Chicago Ave
1,2,Au Cheval,Fried Bologna,9.0,800 W. Randolph St
2,3,Xoco,Woodland Mushroom,9.5,445 N. Clark St
3,4,Al’s Deli,Roast Beef,9.4,914 Noyes St
4,5,Publican Quality Meats,PB&L,10.0,825 W. Fulton Mkt


In [20]:
cursor.execute("select * from sandwich where cafe like ('%Kitchen%') or cafe like ('%Cafe%') order by price")

result = cursor.fetchall()
for row in result:
    print(row)



(45, 'Cafecito', 'Cubana', 5.49, ' 26 E. Congress Pkwy')
(22, 'Dawali Mediterranean Kitchen', 'Shawarma', 6.0, 'Multiple location')
(41, 'Z&H MarketCafe', 'The Marty ', 7.25, ' 1323 E. 57th St')
(35, 'Floriole Cafe & Bakery', 'B.A.D.', 9.0, '1220 W. Webster Ave')
(36, 'First Slice Pie Café', 'Duck Confit and Mozzarella', 9.0, '5357 N. Ashland Ave')
(8, 'Birchwood Kitchen', 'Smoked Salmon Salad', 10.0, '2211 W. North Ave')
(14, 'Crosby’s Kitchen', 'Chicken Dip', 10.0, '3455 N. Southport Ave')
(11, 'Lula Cafe', 'Ham and Raclette Panino', 11.0, '2537 N. Kedzie Blvd')


In [21]:
# 컬럼 이름 지정
columns = ['rand', 'cafe', 'menu', 'price', 'address']

# 데이터프레임 생성
df_kitchen = pd.DataFrame(result, columns=columns)
df_kitchen.head()

,rand,cafe,menu,price,address
0,45,Cafecito,Cubana,5.49,26 E. Congress Pkwy
1,22,Dawali Mediterranean Kitchen,Shawarma,6.00,Multiple location
2,41,Z&H MarketCafe,The Marty,7.25,1323 E. 57th St
3,35,Floriole Cafe & Bakery,B.A.D.,9.00,1220 W. Webster Ave
4,36,First Slice Pie Café,Duck Confit and Mozzarella,9.00,5357 N. Ashland Ave


In [23]:
cursor.execute("select * from sandwich where menu like ('%Salad%') or menu like ('%Cheese%') or menu like ('%Beef%') order by price")

result = cursor.fetchall()
for row in result:
    print(row)


df_kitchen = pd.DataFrame(result, columns=columns)
df_kitchen.head()

(47, 'The Goddess and Grocer', 'Debbie’s Egg Salad', 6.5, ' 25 E. Delaware Pl')
(6, 'Hendrickx Belgian Bread Crafter', 'Belgian Chicken Curry Salad', 7.25, ' 100 E. Walton St')
(48, 'Zenwich', 'Beef Curry', 7.5, ' 416 N. York St')
(23, 'Big Jones', 'Toasted Pimiento Cheese', 8.0, '5347 N. Clark St')
(4, 'Al’s Deli', 'Roast Beef', 9.4, ' 914 Noyes St')
(32, 'La Fournette', 'Tuna Salad', 9.75, ' 1547 N. Wells St')
(31, 'Hannah’s Bretzel', 'Serrano Ham and Manchego Cheese', 9.79, ' Multiple location')
(8, 'Birchwood Kitchen', 'Smoked Salmon Salad', 10.0, '2211 W. North Ave')
(17, 'Manny’s', 'Corned Beef', 11.95, ' 1141 S. Jefferson St')


,rand,cafe,menu,price,address
0,47,The Goddess and Grocer,Debbie’s Egg Salad,6.50,25 E. Delaware Pl
1,6,Hendrickx Belgian Bread Crafter,Belgian Chicken Curry Salad,7.25,100 E. Walton St
2,48,Zenwich,Beef Curry,7.50,416 N. York St
3,23,Big Jones,Toasted Pimiento Cheese,8.00,5347 N. Clark St
4,4,Al’s Deli,Roast Beef,9.40,914 Noyes St


In [24]:
df = pd.read_csv('../../data/movie.csv', encoding='UTF-8-SIG')
df.head()

,date,name,point
0,2017-12-01,뷰티풀 투모로우,9.36
1,2017-12-01,아이 캔 스피크,9.34
2,2017-12-01,다시 태어나도 우리,9.32
3,2017-12-01,록키,9.31
4,2017-12-01,저수지 게임,9.29


In [25]:
cur = remote.cursor()
cur.execute("CREATE TABLE movie (date date, name varchar(64),point float)")#숫자로 시작하는 열 이름을 backtick (`)로 감싸야 합니다.

In [27]:
sql = """insert into movie values (%s,%s,%s)""" #따옴표를 세 개씩 (""") 사용하는 것은 여러 줄에 걸친 문자열을 정의하는 Python의 표준 문법
cursor = remote.cursor(buffered=True)


In [28]:
for i , row in df.iterrows(): #iterrows()는 DataFrame의 각 행을 (인덱스, 행 데이터)의 튜플로 반환
    cursor.execute(sql, tuple(row)) # execute 메서드를 호출하여 SQL 쿼리를 실행하고, 튜플의 값이 쿼리에 매개변수로 전달되어 데이터베이스에 레코드를 추가
    print(tuple(row))
    remote.commit()

('2017-12-01', '뷰티풀 투모로우', 9.36)
('2017-12-01', '아이 캔 스피크', 9.34)
('2017-12-01', '다시 태어나도 우리', 9.32)
('2017-12-01', '록키', 9.31)
('2017-12-01', '저수지 게임', 9.29)
('2017-12-01', '땐뽀걸즈', 9.25)
('2017-12-01', '나, 다니엘 블레이크', 9.22)
('2017-12-01', '러빙 빈센트', 9.21)
('2017-12-01', '스쿨 오브 락', 9.21)
('2017-12-01', '킹 오브 프리즘 프라이드 더 히어로', 9.19)
('2017-12-01', '내 사랑', 9.18)
('2017-12-01', '범죄도시', 9.17)
('2017-12-01', '공범자들', 9.16)
('2017-12-01', '댄서', 9.13)
('2017-12-01', '히트', 9.11)
('2017-12-01', '이프 온리', 9.08)
('2017-12-01', '버킷리스트: 죽기 전에 꼭 하고 싶은 것들', 9.08)
('2017-12-01', '채비', 9.04)
('2017-12-01', '조제, 호랑이 그리고 물고기들', 9.01)
('2017-12-01', '원스', 8.99)
('2017-12-01', '빅풋 주니어', 8.92)
('2017-12-01', '매디슨 카운티의 다리', 8.91)
('2017-12-01', '행복을 찾아서', 8.9)
('2017-12-01', '10분', 8.89)
('2017-12-01', '여배우는 오늘도', 8.88)
('2017-12-01', '벤자민 버튼의 시간은 거꾸로 간다', 8.87)
('2017-12-01', '토르: 라그나로크', 8.87)
('2017-12-01', '넛잡 2', 8.83)
('2017-12-01', '순정', 8.77)
('2017-12-01', '김광석', 8.77)
('2017-12-01', '행복한 사전', 8.67)
('20

In [67]:
cursor.execute("select * from movie")

result = cursor.fetchall()
for row in result:
    print(row)


# 컬럼 이름 지정
columns = ['date', 'name', 'point']

df_movie = pd.DataFrame(result, columns=columns)
df_movie.head()

(datetime.date(2017, 12, 1), '뷰티풀 투모로우', 9.36)
(datetime.date(2017, 12, 1), '아이 캔 스피크', 9.34)
(datetime.date(2017, 12, 1), '다시 태어나도 우리', 9.32)
(datetime.date(2017, 12, 1), '록키', 9.31)
(datetime.date(2017, 12, 1), '저수지 게임', 9.29)
(datetime.date(2017, 12, 1), '땐뽀걸즈', 9.25)
(datetime.date(2017, 12, 1), '나, 다니엘 블레이크', 9.22)
(datetime.date(2017, 12, 1), '러빙 빈센트', 9.21)
(datetime.date(2017, 12, 1), '스쿨 오브 락', 9.21)
(datetime.date(2017, 12, 1), '킹 오브 프리즘 프라이드 더 히어로', 9.19)
(datetime.date(2017, 12, 1), '내 사랑', 9.18)
(datetime.date(2017, 12, 1), '범죄도시', 9.17)
(datetime.date(2017, 12, 1), '공범자들', 9.16)
(datetime.date(2017, 12, 1), '댄서', 9.13)
(datetime.date(2017, 12, 1), '히트', 9.11)
(datetime.date(2017, 12, 1), '이프 온리', 9.08)
(datetime.date(2017, 12, 1), '버킷리스트: 죽기 전에 꼭 하고 싶은 것들', 9.08)
(datetime.date(2017, 12, 1), '채비', 9.04)
(datetime.date(2017, 12, 1), '조제, 호랑이 그리고 물고기들', 9.01)
(datetime.date(2017, 12, 1), '원스', 8.99)
(datetime.date(2017, 12, 1), '빅풋 주니어', 8.92)
(datetime.date(2017, 12, 1), '

,date,name,point
0,2017-12-01,뷰티풀 투모로우,9.36
1,2017-12-01,아이 캔 스피크,9.34
2,2017-12-01,다시 태어나도 우리,9.32
3,2017-12-01,록키,9.31
4,2017-12-01,저수지 게임,9.29


In [85]:
cursor.execute("select * from movie where date >= 20180101 and point between 5 and 6 order by point desc,name")

result = cursor.fetchall()


df_movie = pd.DataFrame(result, columns=columns)
# df_movie

df_movie.iloc[29]

date     2018-02-17
name             흥부
point          5.69
Name: 29, dtype: object

In [48]:
df = pd.read_csv('../../data/def.csv', encoding='UTF-8-SIG')
df.head()

,코드,명칭,주소,전화번호,영업시간,재고량
0,A0013146,현풍(마산방향),대구 달성군 현풍읍 성하길 48,053-614-2437,매일 24시간,1300
1,A0010223,언양(서울방향),울산 울주군 언양읍 경부고속도로 44,052-263-6146,매일 24시간,4496
2,A0033516,울주(울산방향),울산 울주군 삼동면 보은리 산213-15,052-977-2243,매일 24시간,3024
3,A0033517,울주(함양방향),울산 울주군 삼동면 보은리 산192-3,052-977-3243,매일 24시간,2571
4,A0033432,구리(일산방향),경기도 구리시 수도권 제1순환도로 32,031-555-0740,매일 24시간,112


In [49]:
cur = remote.cursor()
cur.execute("CREATE TABLE def ( code char(8), name varchar(16),address varchar(64), phone varchar(16), opentime varchar(16),inventory int)")#숫자로 시작하는 열 이름을 backtick (`)로 감싸야 합니다.

In [51]:
sql = """insert into def values (%s,%s,%s,%s,%s,%s)""" #따옴표를 세 개씩 (""") 사용하는 것은 여러 줄에 걸친 문자열을 정의하는 Python의 표준 문법
cursor = remote.cursor(buffered=True)


In [52]:
for i , row in df.iterrows(): #iterrows()는 DataFrame의 각 행을 (인덱스, 행 데이터)의 튜플로 반환
    cursor.execute(sql, tuple(row)) # execute 메서드를 호출하여 SQL 쿼리를 실행하고, 튜플의 값이 쿼리에 매개변수로 전달되어 데이터베이스에 레코드를 추가
    print(tuple(row))
    remote.commit()

('A0013146', '현풍(마산방향)', '대구 달성군 현풍읍 성하길 48', '053-614-2437', '매일 24시간', 1300)
('A0010223', '언양(서울방향)', '울산 울주군 언양읍 경부고속도로 44', '052-263-6146', '매일 24시간', 4496)
('A0033516', '울주(울산방향)', '울산 울주군 삼동면 보은리 산213-15', '052-977-2243', '매일 24시간', 3024)
('A0033517', '울주(함양방향)', '울산 울주군 삼동면 보은리 산192-3', '052-977-3243', '매일 24시간', 2571)
('A0033432', '구리(일산방향)', '경기도 구리시 수도권 제1순환도로 32', '031-555-0740', '매일 24시간', 112)
('A0003399', '안성(서울방향)', '경기도 안성시 원곡면 경부고속도로 372', '031-664-4096', '매일 24시간', 4200)
('A0004088', '이천(통영방향)', '경기도 이천시 마장면 중부대로609번길 244-35', '031-637-0401', '매일 24시간', 138)
('A0033485', '평택(제천방향)', '경기도 평택시 청북읍 평택제천고속도로 6', '070-4172-8087', '매일 24시간', 2380)
('A0033482', '평택(평택방향)', '경기도 평택시 청북읍 평택제천고속도로 6', '070-4172-8085', '매일 24시간', 1950)
('A0002561', '화성(목포방향)', '경기도 화성시 팔탄면 서해안고속도로 301', '031-353-8015', '매일 24시간', 1150)
('A0009013', '송산포도(시흥방향)', '경기도 화성시 송산면 평택시흥고속도로 25', '031-357-8192', '매일 24시간', 650)
('A0009012', '송산포도(평택방향)', '경기도 화성시 송산면 평택시흥고속도로 25', '031-357-8194', '매일 24

In [55]:
cursor.execute("select * from def")

result = cursor.fetchall()
for row in result:
    print(row)

# 컬럼 이름 지정
columns = ['코드', '명칭', '주소', '전화번호','영업시간','재고량']

df_def = pd.DataFrame(result,columns=columns)
df_def.head()

('A0013146', '현풍(마산방향)', '대구 달성군 현풍읍 성하길 48', '053-614-2437', '매일 24시간', 1300)
('A0010223', '언양(서울방향)', '울산 울주군 언양읍 경부고속도로 44', '052-263-6146', '매일 24시간', 4496)
('A0033516', '울주(울산방향)', '울산 울주군 삼동면 보은리 산213-15', '052-977-2243', '매일 24시간', 3024)
('A0033517', '울주(함양방향)', '울산 울주군 삼동면 보은리 산192-3', '052-977-3243', '매일 24시간', 2571)
('A0033432', '구리(일산방향)', '경기도 구리시 수도권 제1순환도로 32', '031-555-0740', '매일 24시간', 112)
('A0003399', '안성(서울방향)', '경기도 안성시 원곡면 경부고속도로 372', '031-664-4096', '매일 24시간', 4200)
('A0004088', '이천(통영방향)', '경기도 이천시 마장면 중부대로609번길 244-35', '031-637-0401', '매일 24시간', 138)
('A0033485', '평택(제천방향)', '경기도 평택시 청북읍 평택제천고속도로 6', '070-4172-8087', '매일 24시간', 2380)
('A0033482', '평택(평택방향)', '경기도 평택시 청북읍 평택제천고속도로 6', '070-4172-8085', '매일 24시간', 1950)
('A0002561', '화성(목포방향)', '경기도 화성시 팔탄면 서해안고속도로 301', '031-353-8015', '매일 24시간', 1150)
('A0009013', '송산포도(시흥방향)', '경기도 화성시 송산면 평택시흥고속도로 25', '031-357-8192', '매일 24시간', 650)
('A0009012', '송산포도(평택방향)', '경기도 화성시 송산면 평택시흥고속도로 25', '031-357-8194', '매일 24

,코드,명칭,주소,전화번호,영업시간,재고량
0,A0013146,현풍(마산방향),대구 달성군 현풍읍 성하길 48,053-614-2437,매일 24시간,1300
1,A0010223,언양(서울방향),울산 울주군 언양읍 경부고속도로 44,052-263-6146,매일 24시간,4496
2,A0033516,울주(울산방향),울산 울주군 삼동면 보은리 산213-15,052-977-2243,매일 24시간,3024
3,A0033517,울주(함양방향),울산 울주군 삼동면 보은리 산192-3,052-977-3243,매일 24시간,2571
4,A0033432,구리(일산방향),경기도 구리시 수도권 제1순환도로 32,031-555-0740,매일 24시간,112


In [65]:
cursor.execute("select code,name,address, inventory from def where address like ('%고속도로%') and (name like ('%부산방향%') or name like ('%서울방향%')) order by inventory desc")

result = cursor.fetchall()


df_def = pd.DataFrame(result)
df_def

,0,1,2,3
0,A0010223,언양(서울방향),울산 울주군 언양읍 경부고속도로 44,4496
1,A0033068,건천(부산방향),경북 경주시 건천읍 경부고속도로 77,4269
2,A0003399,안성(서울방향),경기도 안성시 원곡면 경부고속도로 372,4200
3,A0017711,군산(서울방향),전북 군산시 나포면 서해안고속도로 148,3970
4,A0032083,통도사(부산방향),경남 양산시 하북면 경부고속도로 31-1,2796
5,A0025491,김천(서울방향),경북 김천시 농소면경부고속도로194,2780
6,A0025482,추풍령(부산방향),경북 김천시 봉산면 경부고속도로 213,2545
7,A0019045,고창고인돌(서울방향),전북 고창군 고창읍 서해안고속도로 80-1,1722
8,A0025495,김천(부산방향),경북 김천시 농소면 경부고속도로 193,1000
9,A0032977,진주(부산방향),경남 진주시 남해고속도로 194,200
